In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from joblib import dump,load

In [2]:
#读数据标准化划分数据集
df = pd.read_csv('../my_data.csv')
df

,Formula,K_frac,Rb_frac,Cs_frac,MA_frac,FA_frac,Li_frac,Ca_frac,Sr_frac,Ba_frac,...,X_Heat_of_fusion(kJ/mol),A_Heat_of_vap(kJ/mol),B_Heat_of_vap(kJ/mol),X_Heat_of_vap(kJ/mol),A_Electronegativity,B_Electronegativity,X_Electronegativity,Decomposition Energy (eV p.f.u.),Band Gap (eV),SLME @ 5um
0,MAGeCl_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.21,23.85,334.30,10.20,2.55,2.01,3.16,-0.391,1.954,0.148
1,MAGeBr_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.29,23.85,334.30,14.73,2.55,2.01,2.96,-0.243,1.612,0.204
2,MAGeI_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,7.76,23.85,334.30,20.90,2.55,2.01,2.66,-0.030,1.311,0.231
3,MASnCl_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,3.21,23.85,290.37,10.20,2.55,1.96,3.16,-0.246,1.582,0.165
4,MASnBr_3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,5.29,23.85,290.37,14.73,2.55,1.96,2.96,-0.228,1.262,0.198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,FAGeI_1.125_Cl_1.875,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,4.92,29.00,334.30,14.21,2.57,2.01,2.97,-1.075,2.156,0.110
484,FASnI_0.375_Cl_2.625,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,3.78,29.00,290.37,11.54,2.57,1.96,3.10,-1.133,2.576,0.062
485,MAPbCl_1.125_I_1.875,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,6.05,23.85,177.90,16.89,2.55,2.33,2.85,-0.003,1.867,0.164
486,MASrCl_0.75_I_2.25,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,6.62,23.85,136.90,18.23,2.55,0.95,2.79,0.101,3.693,0.001


In [3]:
X = df.iloc[:, 1:58]
Y = df.iloc[:, 58:]
Y

,Decomposition Energy (eV p.f.u.),Band Gap (eV),SLME @ 5um
0,-0.391,1.954,0.148
1,-0.243,1.612,0.204
2,-0.030,1.311,0.231
3,-0.246,1.582,0.165
4,-0.228,1.262,0.198
...,...,...,...
483,-1.075,2.156,0.110
484,-1.133,2.576,0.062
485,-0.003,1.867,0.164
486,0.101,3.693,0.001


In [4]:
Y=Y['Decomposition Energy (eV p.f.u.)']
Y

0     -0.391
1     -0.243
2     -0.030
3     -0.246
4     -0.228
       ...  
483   -1.075
484   -1.133
485   -0.003
486    0.101
487   -0.689
Name: Decomposition Energy (eV p.f.u.), Length: 488, dtype: float64

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)
print("X_train's shape is", X_train.shape,"; y_train's shape is", y_train.shape)
print("X_test's shape is", X_test.shape,"; y_test's shape is",y_test.shape)
scaler = StandardScaler()
scaler.fit(X_train)
X_train_stand = scaler.transform(X_train)
X_test_stand = scaler.transform(X_test)

X_train's shape is (390, 57) ; y_train's shape is (390,)
X_test's shape is (98, 57) ; y_test's shape is (98,)


In [10]:
##gridsearch and cv select model##
max_depth = [int(x) for x in np.linspace(10, 130, num =3)]
max_depth.append(None)
param_grid = {'n_estimators': [5,10,20,50,80,90,100,200,500,1000],
                'max_features': ['auto', 'sqrt'],
                'max_depth': max_depth,
                'min_samples_split': [1,2,3,5],
                'min_samples_leaf': [1,2,3],
                'bootstrap': [True]}

In [11]:
rf = RandomForestRegressor()
model = GridSearchCV(estimator =rf,scoring='neg_root_mean_squared_error', param_grid = param_grid,cv=5, verbose=3,n_jobs =-1,return_train_score=True)
model.fit(X_train_stand, y_train)
print(model.cv_results_)
dfresult=pd.DataFrame(model.cv_results_)
dfresult.to_csv('Dec_RF13y32.csv')
print(dfresult)
print(model.best_params_)
print(model.scorer_)
print(model.best_score_)
bestmodel=model.best_estimator_

Fitting 5 folds for each of 960 candidates, totalling 4800 fits


D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
1200 fits failed out of a total of 4800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1200 fits failed with the following error:
Traceback (most recent call last):
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "D:\App\Anaconda3\envs\test\lib\site-packages\sklearn\ensemble\_forest.py", line 467, in fit
    for i, t in enumerate(trees)
  File "D:\App\Anaconda3\envs\test\lib\site-packages\joblib\parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "D:\App\Anaconda3\env

{'mean_fit_time': array([5.18379211e-03, 1.19886875e-02, 8.61201286e-03, 1.57271862e-02,
       2.66608715e-02, 3.05983543e-02, 3.44708443e-02, 6.68153763e-02,
       2.86193895e-01, 7.49758625e-01, 2.10004330e-02, 8.13685894e-02,
       1.76773119e-01, 3.84580612e-01, 6.35310078e-01, 8.86358500e-01,
       8.01492882e-01, 1.73748569e+00, 3.65364709e+00, 6.85660787e+00,
       2.06690788e-02, 6.18006229e-02, 8.96222591e-02, 3.62575722e-01,
       8.31004667e-01, 7.67789173e-01, 8.24056530e-01, 1.52568388e+00,
       4.07380729e+00, 9.03578410e+00, 7.28954315e-02, 9.40923214e-02,
       1.34399509e-01, 4.15901566e-01, 7.18081474e-01, 8.39700985e-01,
       6.32521534e-01, 1.53241167e+00, 3.76321964e+00, 6.96721172e+00,
       3.25384140e-03, 1.65999889e-02, 8.60176086e-03, 2.72634983e-02,
       6.49801254e-02, 6.25422478e-02, 7.32014179e-02, 1.23499107e-01,
       2.81384373e-01, 6.41714811e-01, 2.29990005e-02, 5.47116756e-02,
       1.25101471e-01, 3.56051779e-01, 6.61324644e-01, 8.79